In [116]:
# -*- coding: utf-8 -*-
#
# Birdsong classificatione in noisy environment with convolutional neural nets in Keras
# Copyright (C) 2017 Báint Czeba, Bálint Pál Tóth (toth.b@tmit.bme.hu)
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>. (c) Balint Czeba, Balint Pal Toth
# 
# Please cite the following paper if this code was useful for your research:
# 
# Bálint Pál Tóth, Bálint Czeba,
# "Convolutional Neural Networks for Large-Scale Bird Song Classification in Noisy Environment", 
# In: Working Notes of Conference and Labs of the Evaluation Forum, Évora, Portugália, 2016, p. 8

# this script is responsible for training the neural networks

from scipy import io
import pandas as pd
import numpy as np
import time
import pickle
import os
import h5py
import sys, getopt
import datetime

import matplotlib.pyplot as plt


In [117]:
if __name__ == "__main__":
    argv=sys.argv[1:]


In [118]:
# nb_epochs	= 1 # number of epochs, should be high, the end of the learning process is controled by early stoping
epochs = 20000
# nb_epochs	= 20000 # number of epochs, should be high, the end of the learning process is controled by early stoping
es_patience	= 100 # patience for early stoping 
batchSize	= 1350 # batch size for mini-batch training
# batchSize	= 1 # batch size for mini-batch training
###################################################################################################
# hdf5path	= '../birdclef_data/data_top999_nozero.hdf5' # training data generated by loadData.py
hdf5path	= '../birdclef_data/data_top468_nozero.hdf5' # training data generated by loadData.py
##################################################################################################
# modelPath	= './model-AlexNet.py' # filename of the model to use (currently model-birdClef.py or model-AlexNet.py)
modelPath	= '../model-AlexNet.py' # filename of the model to use (currently model-birdClef.py or model-AlexNet.py)
logfileName	= 'log.xls'
#scalerFilePath	= '../birdclef_data/standardScaler_5000.pickle'
scalerFilePath	= None
preTrainedModelWeightsPath = None # path and filename to pretrained network: if there is a pretrained network, we can load it and continue to train it
# tensorflowBackend = False # set true if Keras has TensorFlow backend - this way we set TF not to allocate all the GPU memory
tensorflowBackend = True # set true if Keras has TensorFlow backend - this way we set TF not to allocate all the GPU memory

In [119]:
os.environ["MKL_THREADING_LAYER"] = "GNU"

if (tensorflowBackend):
	import tensorflow as tf
	config = tf.ConfigProto()
	config.gpu_options.allow_growth=True
	sess = tf.Session(config=config)
	from keras import backend as K
	K.set_session(sess)  #這裡有問題，沒這參數


In [120]:
print('epochs: %d, hdf5path: %s, scalerFilePath: %s' % (epochs, hdf5path,scalerFilePath))

epochs: 20000, hdf5path: ../birdclef_data/data_top468_nozero.hdf5, scalerFilePath: None


In [121]:
scaler = None
scaleData = None
# if a scaler file generated by loadData.py is given, than load it and define a scaler function that will be used later
if scalerFilePath is not None:
    scaler = pickle.load(open(scalerFilePath, 'rb'))
    # Can't use scaler.transform because it only supports 2d arrays.
    def scaleData(X):
        return (X-scaler.mean_)/scaler.scale_


In [122]:
from io_utils_mod import HDF5Matrix
f = h5py.File(hdf5path, 'r')
X = f.get('X')
y = f.get('y')
print("Shape of X: ")
print(X.shape)
dataSetLength	= X.shape[0]
output_dim	= y.shape[1] #len(y_train[0])
# test and validation splits
testSplit	= 0.01 # 1%
validationSplit	= 0.05 # 5%
# validationSplit	= 0.1 # 5%
f.close()


Shape of X: 
(5530, 1, 200, 310)


In [123]:
output_dim

468

In [124]:
# load training data
X_train = HDF5Matrix(hdf5path, 'X', 0, int(dataSetLength*(1-(testSplit+validationSplit))), normalizer = scaleData) 
y_train = HDF5Matrix(hdf5path, 'y', 0, int(dataSetLength*(1-(testSplit+validationSplit))))
# load validation data
X_validation = HDF5Matrix(hdf5path, 'X', int(dataSetLength*(1-(testSplit+validationSplit)))+1, 
                          int(dataSetLength*(1-testSplit)), normalizer = scaleData)
y_validation = HDF5Matrix(hdf5path, 'y', int(dataSetLength*(1-(testSplit+validationSplit)))+1, 
                          int(dataSetLength*(1-testSplit)))
# load test data
X_test = HDF5Matrix(hdf5path, 'X', int(dataSetLength*(1-testSplit))+1, dataSetLength, normalizer = scaleData)
y_test = HDF5Matrix(hdf5path, 'y', int(dataSetLength*(1-testSplit))+1, dataSetLength)

In [125]:
y_train.data

<HDF5 dataset "y": shape (5530, 468), type "<f4">

In [126]:
print("Shape of X_train after train-validation-test split:")
print(X_train.shape)

Shape of X_train after train-validation-test split:
(5198, 1, 200, 310)


In [127]:
X_train[2]

array([[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ..., 
        [ 0.23109023,  1.11156821,  0.01434718, ...,  1.48071444,
          1.52842546,  1.34299016],
        [ 0.14459638,  0.99199986,  0.38565734, ...,  0.09228765,
          1.13456285, -0.36999109],
        [ 1.06557512,  0.71072274,  0.93754435, ..., -0.67539954,
          0.2943573 ,  0.78111243]]], dtype=float32)

In [128]:

import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import BatchNormalization
from keras.layers import Conv2D , MaxPooling2D
from keras.optimizers import SGD, RMSprop
# from keras.layers.recurrent import LSTM
from keras.callbacks import EarlyStopping, ModelCheckpoint
from MapCallback import MapCallback
# from keras.layers.convolutional import Convolution2D,  MaxPooling2D 

# from theano import ifelse
# Strongly AlexNet based convolutional neural network



In [129]:
model = Sequential()

# data_format="channels_first"
input_shape = (1,200,310)

In [133]:
# ?Conv2D
# Init signature: Conv2D(filters, kernel_size, strides=(1, 1), padding='valid', data_format=None, dilation_rate=(1, 1), 
#                        activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', 
#                        kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, 
#                        bias_constraint=None, **kwargs)

In [139]:
# # 卷积层一
# model.add(Conv2D(32, kernel_size = (5, 5), strides = (1, 1), padding = 'same', activation = 'relu', input_shape = (1, 28, 28)))
# #model.add(Convolution2D(
# #    nb_filter=32,
# #    nb_row=5,
# #    nb_col=5,
# #    border_mode='same',     # Padding method
# #    dim_ordering='th',    # 采用 theano 的 input 格式  
# #    input_shape=(1,         # channels
# #                 28, 28,)    # height & width
# #))
# #model.add(Activation('relu'))

# 作者：Ledestin
# 链接：https://www.jianshu.com/p/ddf5ed31e44c
# 來源：简书
# 著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

In [141]:
# convolutional layer
# model.add(Conv2D(input_shape=(1,200,310), 
#                  data_format="channels_first",
#                  nb_filter=48*2,
#                  nb_row=16,
#                  nb_col=16,
#                  border_mode='valid',
#                  init='glorot_normal', #glorot_normal lecun_uniform he_uniform
#                  activation='relu'
#                 ))

model.add(Conv2D(96, kernel_size = (5,5),  padding = 'same', activation = 'relu', input_shape = (-1,1,200,310)))

model.add(BatchNormalization())                        
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(nb_filter=128*2,
                        nb_row=3,
                        nb_col=3,
                        border_mode='valid',
                        init='lecun_uniform', #glorot_normal lecun_uniform he_uniform
                        activation='relu'
                        ))  
model.add(BatchNormalization())                        
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(nb_filter=192*2,
                        nb_row=3,
                        nb_col=3,
                        border_mode='same',
                        init='lecun_uniform', #glorot_normal lecun_uniform he_uniform
                        activation='relu'
                        ))  

model.add(Conv2D(nb_filter=192*2,
                        nb_row=3,
                        nb_col=3,
                        border_mode='same',
                        init='lecun_uniform', #glorot_normal lecun_uniform he_uniform
                        activation='relu'
                        ))  

model.add(Conv2D(nb_filter=128*2,
                        nb_row=3,
                        nb_col=3,
                        border_mode='same',
                        init='lecun_uniform', #glorot_normal lecun_uniform he_uniform
                        activation='relu'
                        ))  

model.add(BatchNormalization())                        
model.add(MaxPooling2D(pool_size=(2,2)))

# dense layers                  
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(output_dim = output_dim, activation='softmax'))

ValueError: Input 0 is incompatible with layer conv2d_26: expected ndim=4, found ndim=2

In [ ]:
model.summary()

In [ ]:
# load model and compile it, we use RMSprop here, other optimizer algorithm should be tested
# execfile(modelPath)
# exec(open('model-AlexNet.py', encoding="utf-8").read())
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')#, metrics=["accuracy"])


In [ ]:
# print the model
print("The following model is used: ")
for layer in model.layers:
    print("{} output shape: {}".format(layer.name, layer.output_shape))

In [ ]:
# load pretrained model if it is set
if preTrainedModelWeightsPath is not None:
    model.load_weights(preTrainedModelWeightsPath)
    print("Reloaded weights from: {}".format(preTrainedModelWeightsPath))

In [ ]:
# define callback functions
mapcallback	= MapCallback()
earlyStopping	= EarlyStopping(monitor='val_loss', patience = es_patience) # early stoping
# save best models based on accuracy, loss and MAP metrics
#bestModelFilePath_val_map	= './modelWeights/best_val_map_{}_{}.hdf5'.format(output_dim, datetime.datetime.now().strftime('%Y-%m-%d-%M-%S'))
#bestModelFilePath_val_acc	= './modelWeights/best_val_acc_{}_{}.hdf5'.format(output_dim, datetime.datetime.now().strftime('%Y-%m-%d-%M-%S'))
#bestModelFilePath_val_loss	= './modelWeights/best_val_loss_{}_{}.hdf5'.format(output_dim, datetime.datetime.now().strftime('%Y-%m-%d-%M-%S'))
bestModelFilePath_val_acc	= './modelWeights/best_val_acc_{}.hdf5'.format(output_dim)
bestModelFilePath_val_loss	= './modelWeights/best_val_loss_{}.hdf5'.format(output_dim)
bestModelFilePath_val_map	= './modelWeights/best_val_map_{}.hdf5'.format(output_dim)
checkpointer_val_acc	= ModelCheckpoint(filepath = bestModelFilePath_val_acc, verbose = 1, monitor = 'val_acc', save_best_only = True)
checkpointer_val_loss	= ModelCheckpoint(filepath = bestModelFilePath_val_loss, verbose = 1, monitor = 'val_loss', save_best_only = True)
checkpointer_val_map	= ModelCheckpoint(filepath = bestModelFilePath_val_map, verbose = 1, monitor = 'val_map', mode = 'max', save_best_only = True)


In [ ]:
# # store the starting time 
startTime = time.time()

In [ ]:
# training
fitting_result = model.fit(X_train, y_train, epochs = epochs, batch_size = batchSize,
                            callbacks = [earlyStopping,
                                         mapcallback,
                                         checkpointer_val_acc, 
                                         checkpointer_val_loss,  
                                         checkpointer_val_map
                                        ], 
                            shuffle = None, 
                            validation_data = (X_validation, y_validation)
                           )






# calculate the elapsed time
elapsed = time.time()-startTime;
print("Execution time: {0} s".format(elapsed))


In [ ]:
# ?model.fit
# Signature: model.fit(x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None, validation_split=0.0, 
# validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, 
# validation_steps=None, **kwargs)
# Docstring:
# Trains the model for a fixed number of epochs (iterations on a dataset).

In [ ]:
# # store the starting time 
# startTime = time.time()

# # # fitting_result = model.fit(X_train, y_train, epochs = epochs, batch_size = batchSize, callbacks = [earlyStopping, mapcallback, checkpointer_val_acc, checkpointer_val_loss,  checkpointer_val_map], shuffle = 'batch', validation_data = (X_validation, y_validation))
# fitting_result = model.fit(X_train, y_train, epochs = epochs, batch_size = batchSize, 
#                            callbacks = [earlyStopping
# #                                         checkpointer, 
# #                                         lr
#                                        ], shuffle = 'batch'
# #                            validation_data = (X_validation, y_validation)
#                           )

# # calculate the elapsed time
# elapsed = time.time()-startTime;
# print("Execution time: {0} s".format(elapsed))

# # save model
# model.save('train_Model.h5')

In [ ]:
# # calculate the elapsed time
# elapsed = time.time()-startTime;
# print("Execution time: {0} s".format(elapsed))


In [ ]:
# convert the output (probabilistics) to classes
def proba_to_class(a):
    classCount	= len(a[0])
    to_return	= np.empty((0,classCount))
    for row in a:
        maxind	= np.argmax(row)
        to_return = np.vstack((to_return, [1 if i == maxind else 0 for i in range(classCount)]))
    return to_return


In [ ]:
# calculate metrics on test data with the last model 
from sklearn.metrics import average_precision_score, accuracy_score
y_result	= model.predict(X_test)
map		= average_precision_score( y_test.data[y_test.start: y_test.end], y_result, average='micro')
accuracy	= accuracy_score(y_test.data[y_test.start: y_test.end], proba_to_class(y_result))
print("AveragePrecision: {}".format(map))
print("Accuracy: {}".format(accuracy))


In [ ]:
# reload the best model with smallest validation loss and calculate metrics on test data
print("----- Loading best model from: {}  -------".format(bestModelFilePath_val_loss))
model.load_weights(bestModelFilePath_val_loss)
y_result_bm		= model.predict(X_test)
map_bm_val_loss		= average_precision_score( y_test.data[y_test.start: y_test.end], y_result_bm, average='macro')
accuracy_bm_val_loss	= accuracy_score(y_test.data[y_test.start: y_test.end], proba_to_class(y_result_bm))
print("AveragePrecision: {}".format(map_bm_val_loss))
print("Accuracy: {}".format(accuracy_bm_val_loss))

In [ ]:
# reload the best model with highest validation accuracy and calculate metrics on test data
print("----- Loading best model from: {}  -------".format(bestModelFilePath_val_acc))
model.load_weights(bestModelFilePath_val_acc)
y_result_bm		= model.predict(X_test)
map_bm_val_acc		= average_precision_score( y_test.data[y_test.start: y_test.end], y_result_bm, average='macro')
accuracy_bm_val_acc	= accuracy_score(y_test.data[y_test.start: y_test.end], proba_to_class(y_result_bm))
print("AveragePrecision: {}".format(map_bm_val_acc))
print("Accuracy: {}".format(accuracy_bm_val_acc))

In [ ]:
# save the results summery into an excel file
import log
log.logToXLS(logfileName, model, fitting_result, {'execution(s)':elapsed, 'map':map, 'accuracy':accuracy, 'map_bm_val_loss':map_bm_val_loss, 'accuracy_bm_val_loss':accuracy_bm_val_loss,'map_bm_val_acc':map_bm_val_acc, 'accuracy_bm_val_acc':accuracy_bm_val_acc, 'modelPyFile': modelPath})
